<a id="primero"></a>
## 1. Predicción de Entalpía de Atomización


Las simulaciones de propiedades moleculares son computacionalmente costosas y requieren de un arduo trabajo científico. El objetivo de esta sección corresponde a la utilización de métodos de aprendizaje automático supervisado (Redes Neuronales Artificiales) para predecir propiedades moleculares, en este caso la Energía de Atomización o Entalpía de Atomización, a partir de una base de datos de simulaciones obtenida mediante __[Quantum Espresso](http://www.quantum-espresso.org/)__. Si esto se lograse hacer con gran precisión, se abrirían muchas posibilidades en el diseño computacional y el descubrimiento de nuevas moléculas, compuestos y fármacos.

<img src="https://pubs.rsc.org/services/images/RSCpubs.ePlatform.Service.FreeContent.ImageService.svc/ImageService/Articleimage/2012/NR/c2nr11543c/c2nr11543c-f4.gif" title="Title text" width="40%"/>


La **entalpía de atomización** es la cantidad de variación de entalpía cuando los enlaces de un compuesto se rompen y los componentes se reducen a átomos individuales. Tal como se ha indicado, su tarea es la de predecir dicho nivel a partir de los atributos enunciados en el dataset puesto a vuestra disposición en *moodle*.

In [ ]:
import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta
from keras.regularizers import l1,l2

import matplotlib.pyplot as plt

from ipywidgets import interact

#### **NOTA:**
Debido a lo mucho que toma realizar los experimentos, los errores se guardan en archivos caché `.npy`, estos deben borrarse si se quiere repetir los experimentos.

---
a) Construya un *dataframe* con los datos a analizar y descríbalo brevemete. Además, realice la división de éste en los conjuntos de entrenamiento, validación y testeo correspondientes. Comente por qué se deben eliminar ciertas columnas.

In [ ]:
datos = pd.read_csv("roboBohr.csv")
print("datos.shape:",datos.shape)
datos.info()
datos.describe()

In [ ]:
datos.head()

In [ ]:
datos.drop(columns=['Unnamed: 0','pubchem_id'],axis=1,inplace=True)
total = len(datos)
df_trai = datos[:int(0.6*total)]                       #60% de los datos
df_vali = datos[int(0.6*total):int(0.85*total)]        #25% de los datos
df_test = datos[int(0.85*total)::]                     #15% restante

La columna `Unnamed: 0` representa el id del compuesto dentro del dataset, y `pubchem_id` parece ser un id general para identificar el compuesto. Ambas columnas se remueven porque la asignación de estos índices es arbitraria y su valor no debería estar relacionado con el resultado que debe entregar nuestro modelo (la Entalpía de Atomización, correspondiente a la columna `Eat`). Aunque el modelo debería detectar que no hay correlación entre `Eat` y estos atributos, es mejor removerlos para no *confundir* el aprendizaje.

---
a.1) Una buena práctica es la de normalizar los datos antes de trabajar con el modelo. **Explique por qué se aconseja dicho preprocesamiento**

In [ ]:
# Get scaler and scale data
scaler = StandardScaler().fit(df_trai)
X_trai_scaled = pd.DataFrame(scaler.transform(df_trai),columns=df_trai.columns)
X_vali_scaled = pd.DataFrame(scaler.transform(df_vali),columns=df_vali.columns)
X_test_scaled = pd.DataFrame(scaler.transform(df_test),columns=df_test.columns)
# Get targets
y_trai = df_trai.pop('Eat').values.reshape(-1,1)
y_vali = df_vali.pop('Eat').values.reshape(-1,1)
y_test = df_test.pop('Eat').values.reshape(-1,1)
# Remove targets from attributes
X_trai_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_vali_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_test_scaled.drop(columns=['Eat'],axis=1,inplace=True)

Muchos modelos de aprendizaje son suceptibles a la escala de los atributos, por ese motivo, atributos con órdenes de magnitud mayores pueden afectar más a los mismos.
Para hacer el aprendizaje independiente de las unidades de medición en que se presentan estos atributos y evitar un bias del aprendizaje hacia unos por sobre otros, es que se normalizan.

---
b) Muestre en un gráfico el error cuadrático (MSE) para el conjunto de entrenamiento y de pruebas vs número de *epochs* de entrenamiento, para una red *feedforward* de 3 capas, con 256 unidades ocultas y función de activación sigmoidal. Entrene la red usando gradiente descendente estocástico con tasa de aprendizaje (learning rate) 0.01 y 250 epochs de entrenamiento, en el conjunto de entrenamiento y de validación. Comente. Si observara divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento.

In [ ]:
# Create model:
def create_model(activation='sigmoid',initializer='uniform',
                 lr=None,decay=None,optimizer=None,
                 layer1_reg=None,layer2_reg=None,dropout=None,
                 neurons=256):
    # Define model
    model = Sequential()
    model.add(Dense(neurons,input_dim=X_trai_scaled.shape[1],
                    kernel_initializer=initializer,activation=activation,
                    kernel_regularizer=layer1_reg))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(1, kernel_initializer=initializer,activation="linear",
                    kernel_regularizer=layer2_reg))
    # Set optimizer
    if optimizer is None:
        # default values
        if lr is None: lr = 0.01
        if decay is None: decay = 0.0
        optimizer = SGD(lr=lr,decay=decay)
    else:
        assert lr is None and decay is None
    # Compile model with optimizer
    model.compile(optimizer=optimizer,loss='mean_squared_error')
    return model

In [ ]:
def gradual_color(i,total):
    if i<total//2:
        return (2.0*i/float(total),0.0,0.0)
    else:
        return (1.0,2.0*(i-total//2)/float(total),0.0)

class PlotErrors(keras.callbacks.Callback):
    
    def __init__(self,text=None,labels=None,gradual=False,save_name=None):
        self.trai_err = []
        self.vali_err = []
        self.save_name = save_name
        self.loaded = False
        # Load cache
        if os.path.isfile(self.save_name):
            data = np.load(self.save_name)
            self.trai_err = data[0]
            self.vali_err = data[1]
            self.loaded = True
        # Labels
        self.labels = labels
        self.gradual = gradual
        self.trai_title = "Training error v/s epoch"
        self.vali_title = "Validaton error v/s epoch"
        if text:
            self.trai_title = self.trai_title+" for different "+text
            self.vali_title = self.vali_title+" for different "+text
        self.widget = interact(self.display).widget

    def display(self):
        fig, ax = plt.subplots(1,2,figsize=(12,6),sharex=True,sharey=True)
        ax[0].set_ylim(0.01,1000)
        # Legend and colors:
        legc = {}
        # Train error:
        ax[0].set(xlabel='epoch',ylabel='training error',title=self.trai_title)
        for i in range(len(self.trai_err)):
            #
            if self.labels and self.gradual: legc['c'] = gradual_color(i,len(self.labels))
            if self.labels: legc['label'] = self.labels[i]
            #
            x = np.arange(1,1+len(self.trai_err[i]))
            ax[0].semilogy(x,self.trai_err[i],linewidth=2,**legc)
        ax[0].grid()
        if self.labels: ax[0].legend()
        # Validation error
        ax[1].set(xlabel='epoch',ylabel='validation error',title=self.vali_title)
        for i in range(len(self.vali_err)):
            #
            if self.labels and self.gradual: legc['c'] = gradual_color(i,len(self.labels))
            if self.labels: legc['label'] = self.labels[i]
            #
            x = np.arange(1,1+len(self.vali_err[i]))
            ax[1].semilogy(x,self.vali_err[i],linewidth=2,**legc)
        ax[1].grid()
        if self.labels: ax[1].legend()
        #
        plt.show()

    def on_train_begin(self, logs={}):
        self.trai_err.append([])
        self.vali_err.append([])
    
    def on_train_end(self,logs={}):
        if self.save_name:
            trai_err = np.array(self.trai_err)
            u = np.zeros((2,*trai_err.shape))
            u[0] = trai_err
            u[1] = np.array(self.vali_err)
            np.save(self.save_name,u)

    def on_epoch_end(self, epoch, logs={}):
        vali_loss = logs.get('val_loss')
        trai_loss = logs.get('loss')
        self.vali_err[-1].append(vali_loss)
        self.trai_err[-1].append(trai_loss)
        self.widget.update()
        return

In [ ]:
EPOCHS = 100 # Not 250
EXPERIMENTS = 5
BATCH_SIZE = 512

In [ ]:
callback = PlotErrors("experiments",save_name="1b.npy")
if not callback.loaded:
    for i in range(EXPERIMENTS):
        model = create_model(activation="sigmoid")
        model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
            validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
        del model

**TODO Respuesta**

---
c) Repita el paso anterior, utilizado ’**ReLU**’ como función de activación y compare con lo obtenido en b).

In [ ]:
callback = PlotErrors("experiments",save_name="1c.npy")
if not callback.loaded:
    for i in range(EXPERIMENTS):
        initi = keras.initializers.RandomUniform(minval=-1e-5, maxval=1e-5)
        model = create_model(activation="relu",initializer="zeros")
        model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
            validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
        del model

**TODO Respuesta**

Cabe señalar que se debió utilizar un inicializador lineal con valores pequeños, ya que `kernel_initializer="uniform"` resultó en pérdidas `NaN` y `kernel_initializer="zeros"` resulta en experimentos con resultados iguales.

---
d) Repita b) y c) variando la tasa de aprendizaje (*learning rate*) en un rango sensible. Comente. Si observara divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento.

In [ ]:
n_lr = 10
learn_rate = np.arange(1,n_lr+1)/(20.0*n_lr)
print(learn_rate)

In [ ]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("learning rates",[str(x) for x in learn_rate],gradual=True,
                          save_name="1d_%s.npy"%activ)
    if not callback.loaded:
        for lr in learn_rate:
            model = create_model(activation=activ,initializer="zeros",lr=lr) # Notice zeros
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Nótese que para learning rates grandes la pérdida se transformó en nan.

---
e) Entrene los modelos considerados en b) y c) usando *progressive decay*. Compare y comente.

In [ ]:
INIT_LR = 0.02
n_decays = 10
decays = np.logspace(-6,0,n_decays)
print(decays)

In [ ]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("learning decays",[str(x) for x in decays],gradual=True,
                         save_name="1e_%s.npy"%activ)
    if not callback.loaded:
        for dec in decays:
            model = create_model(activation=activ,initializer="zeros",lr=INIT_LR,decay=dec)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

De acuerdo con [esta respuesta](https://stats.stackexchange.com/a/211340), la tasa de aprendizaje dado un *decay* $D$ es:
$$
L_t = L_0 \, \frac{1}{1 +  t \cdot D}
$$

Se colocó una learning rate relativamente alta pero no lo suficiente como para que resulten pérdidas `nan`.

Se requiere realizar varias veces el mismo experimento, comenzar con inicialización zeros no es garantía de que la convergencia de redes al cambiar hiperparámetros será uniforme.

---
f) Entrene los modelos considerados en b) y c) utilizando SGD en mini-*batches*. Experimente con diferentes tamaños del *batch*. Comente.

In [ ]:
n_batches = 8
batch_sizes = np.array(np.round(
    np.linspace(100,X_trai_scaled.shape[0]//2,n_batches)),dtype='int')
print(batch_sizes)

In [ ]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("batch sizes",[str(x) for x in batch_sizes],gradual=True,
                         save_name="1f_%s.npy"%activ)
    if not callback.loaded:
        for bs in batch_sizes:
            model = create_model(activation=activ,initializer="zeros")
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0,
                validation_data=(X_vali_scaled, y_vali), callbacks=[callback],
                batch_size=bs)
            del model

---
g) Entrene los modelos obtenidos en b) y c) utilizando estrategias modernas para adaptar la tasa de aprendizaje. Compare los desempeños de adagrad, adadelta, RMSprop y adam. ¿Se observa en algún caso un mejor resultado final? ¿Se observa en algún caso una mayor velocidad de convergencia sobre el dataset de entrenamiento? ¿Sobre el dataset de validación?

In [ ]:
optimizers = [SGD,Adam,RMSprop,Adagrad,Adadelta]

In [ ]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("optimizers",[x.__name__ for x in optimizers],
                         save_name="1g_%s.npy"%activ)
    if not callback.loaded:
        for opt in optimizers:
            model = create_model(activation=activ,initializer="zeros",optimizer=opt(lr=0.01))
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                    validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

---
h) Entrene los modelos obtenidos en b) y c) utilizando regularizadores $l_1$ y $l_2$ (*weight decay*). Compare los desempeños de prueba obtenidos antes y después de regularizar. Experimente con distintos valores del parámetro de regularización y comente. Además evalúe el efecto de regularizar solo la primera capa *vs* la segunda, comente.

In [ ]:
regs = [l1(0.001),l1(0.01),l1(0.1),l2(0.001),l2(0.01),l2(0.1)]

In [ ]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("regularizers",["l1=%.3f,l2=%.3f "%(x.l1,x.l2) for x in regs],
                         save_name="1h1_%s.npy"%activ)
    if not callback.loaded:
        for reg in regs:
            model = create_model(activation=activ,initializer="zeros",
                                 layer1_reg=reg,layer2_reg=reg)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                    validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

In [ ]:
dual_regs = {"l1_1layer":(l1(0.01),None),
             "l1_2layer":(None,l1(0.01)),
             "l1_both"  :(l1(0.01),l1(0.01)),
             "l2_1layer":(l2(0.01),None),
             "l2_2layer":(None,l2(0.01)),
             "l2_both"  :(l2(0.01),l2(0.01))}

In [ ]:
names = list(dual_regs.keys())
for activ in ("sigmoid","relu"):
    callback = PlotErrors("regularizers",names,
                         save_name="1h2_%s.npy"%activ)
    if not callback.loaded:
        for nam in names:
            model = create_model(activation=activ,initializer="zeros",
                                 layer1_reg=dual_regs[nam][0],layer2_reg=dual_regs[nam][1])
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                    validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

---
i) Entrene los modelos obtenidos en b) y c) utilizando *Dropout*. Compare los desempeños de prueba obtenidos antes y después de regularizar. Experimente con distintos valores del parámetro de regularización y comente.

In [ ]:
dropouts = [0.2,0.4,0.6,0.8]

In [ ]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("dropouts",[str(x) for x in dropouts],
                         save_name="1i_%s.npy"%activ)
    if not callback.loaded:
        for drop in dropouts:
            model = create_model(activation=activ,initializer="zeros",dropout=drop)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                        validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

---
j) Fijando todos los demás hiper-parámetros del modelo definido en b) y en c), utilice validación cruzada con un número de *folds* igual a *K* = 5 y *K*=10 para determinar el mejor valor correspondiente a un parámetro que usted elija (tasa de aprendizaje, número de neuronas, parámetro de regularización, etc) ¿El mejor parámetro para la red con sigmoidal es distinto que para ReLU? ¿Porqué sucede? Además mida el error real del modelo sobre el conjunto de pruebas, compare y concluya.

In [ ]:
neuron_numbers = [64,128,192,256,320]

In [ ]:
# Create the folds
for activation in ("sigmoid","relu"):
    print("Activation %s:"%activation)
    for folds in (5,10):
        print("\tFolds %d:"%folds)
        Xm = X_trai_scaled.values
        ym = y_trai
        for nneurons in neuron_numbers:
            kfold = KFold(n_splits=folds,shuffle=False)
            cvscores = []
            for train, val in kfold.split(X_trai_scaled):
                model = create_model(activation=activ,initializer="zeros",neurons=nneurons)
                model.fit(Xm[train],ym[train], epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE)
                scores = model.evaluate(Xm[val], ym[val], verbose=0)
                cvscores.append(scores)
            mse_cv = np.mean(cvscores)
            mse_test = np.mean(model.evaluate(X_test_scaled,y_test, verbose=0))
            print("\t\t%4d Neurons:  mse_cv: %10.6f  mse_test: %10.6f"%(
                nneurons,mse_cv,mse_test))